In [2]:
import time
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, KFold
import multiprocessing
import os
from scipy import sparse

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn import ensemble

In [4]:
import datetime as dt
from datetime import date
import calendar
from calendar import weekday, day_name

In [5]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [6]:
from patsy import dmatrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp
from sklearn.datasets import make_regression

In [9]:
def get_holidays(fpath):
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.split(" ")[:3] for line in lines]
    lines = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines = pd.to_datetime(lines)
    return pd.DataFrame({"date2":lines})

In [10]:
def get_holiday_names(fpath):
    # holiday_names are holidays + around Black Fridays
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.strip().split(" ")[:4] for line in lines]
    lines_dt = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines_dt = pd.to_datetime(lines_dt)
    lines_hol = [line[3] for line in lines]
    return pd.DataFrame({"date2":lines_dt, "holiday_name":lines_hol})

In [11]:
def preprocess(_df, is_train):
    
    df = _df.copy()
    
    # date
    df['date2'] = pd.to_datetime(df['date'])
    
    # weekday
    df['weekday'] = df.date2.dt.weekday
    df['is_weekend'] = df.date2.dt.weekday.isin([5,6])
    df['is_holiday'] = df.date2.isin(holidays.date2)
    df['is_holiday_weekday'] = df.is_holiday & (df.is_weekend == False)
    df['is_holiday_weekend'] = df.is_holiday &  df.is_weekend

    # bool to int (maybe no meaning)
    df.is_weekend = np.where(df.is_weekend, 1, 0)
    df.is_holiday = np.where(df.is_holiday, 1, 0)
    df.is_holiday_weekday = np.where(df.is_holiday_weekday, 1, 0)
    df.is_holiday_weekend = np.where(df.is_holiday_weekend, 1, 0)
    
    # day, month, year
    df['day'] = df.date2.dt.day
    df['month'] = df.date2.dt.month
    df['year'] = df.date2.dt.year
    
    # around BlackFriday
    df = pd.merge(df, holiday_names, on='date2', how = 'left')
    df.loc[df.holiday_name.isnull(), "holiday_name"] = ""

    around_BlackFriday = ["BlackFridayM3", "BlackFridayM2", "ThanksgivingDay", "BlackFriday",
                          "BlackFriday1", "BlackFriday2", "BlackFriday3"]
    df["around_BlackFriday"] = np.where(df.holiday_name.isin(around_BlackFriday), 
                                        df.holiday_name, "Else")

    return df

In [12]:
train_file = pd.read_csv("data/train.csv", sep=',')
train_file

,date,store_nbr,item_nbr,units
0,2012-01-01,1,1,0
1,2012-01-01,1,2,0
2,2012-01-01,1,3,0
3,2012-01-01,1,4,0
4,2012-01-01,1,5,0
5,2012-01-01,1,6,0
6,2012-01-01,1,7,0
7,2012-01-01,1,8,0
8,2012-01-01,1,9,29
9,2012-01-01,1,10,0


In [13]:
key_file = pd.read_csv("data/key.csv", sep=',')
key_file

,store_nbr,station_nbr
0,1,1
1,2,14
2,3,7
3,4,9
4,5,12
5,6,14
6,7,6
7,8,4
8,9,17
9,10,12


In [14]:
weather = pd.read_csv('data/weather.csv', sep=',')
weather

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,M,36,40,23,0,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,0716,1626,RA,0.0,0.07,28.82,29.91,9.1,23,11.3
2,3,2012-01-01,55,34,45,9,24,36,20,0,0735,1720,,0.0,0.00,29.77,30.47,9.9,31,10.0
3,4,2012-01-01,63,47,55,4,28,43,10,0,0728,1742,,0.0,0.00,29.79,30.48,8.0,35,8.2
4,6,2012-01-01,63,34,49,0,31,43,16,0,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
5,7,2012-01-01,50,33,42,M,26,35,23,0,-,-,,0.0,0.00,29.15,30.54,10.3,32,10.2
6,8,2012-01-01,66,45,M,M,34,46,M,M,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9
7,9,2012-01-01,34,19,27,M,17,23,38,0,-,-,UP,M,T,29.34,30.09,22.8,30,22.5
8,10,2012-01-01,73,53,63,M,55,58,2,0,0723,1738,FG+ FG BR,M,0.00,30.16,30.19,5.1,24,5.5
9,11,2012-01-01,72,48,60,7,54,56,5,0,0724,1737,FG+ FG BR,0.0,0.00,30.15,30.18,4.6,23,4.8


In [15]:
weather['date'] = pd.to_datetime(weather['date'], errors='coerce')
weather

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,M,36,40,23,0,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,0716,1626,RA,0.0,0.07,28.82,29.91,9.1,23,11.3
2,3,2012-01-01,55,34,45,9,24,36,20,0,0735,1720,,0.0,0.00,29.77,30.47,9.9,31,10.0
3,4,2012-01-01,63,47,55,4,28,43,10,0,0728,1742,,0.0,0.00,29.79,30.48,8.0,35,8.2
4,6,2012-01-01,63,34,49,0,31,43,16,0,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
5,7,2012-01-01,50,33,42,M,26,35,23,0,-,-,,0.0,0.00,29.15,30.54,10.3,32,10.2
6,8,2012-01-01,66,45,M,M,34,46,M,M,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9
7,9,2012-01-01,34,19,27,M,17,23,38,0,-,-,UP,M,T,29.34,30.09,22.8,30,22.5
8,10,2012-01-01,73,53,63,M,55,58,2,0,0723,1738,FG+ FG BR,M,0.00,30.16,30.19,5.1,24,5.5
9,11,2012-01-01,72,48,60,7,54,56,5,0,0724,1737,FG+ FG BR,0.0,0.00,30.15,30.18,4.6,23,4.8


In [16]:
weather_file = pd.read_csv("data/weather_processed.csv", sep = ',')
weather_file

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52.0,31.0,42.0,16.0,36.0,40.0,23.0,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
1,2,2012-01-01,48.0,33.0,41.0,16.0,37.0,39.0,24.0,0.0,716.0,1626.0,0.0,0.07,28.82,29.91,9.1,23.0,11.3
2,3,2012-01-01,55.0,34.0,45.0,9.0,24.0,36.0,20.0,0.0,735.0,1720.0,0.0,0.00,29.77,30.47,9.9,31.0,10.0
3,4,2012-01-01,63.0,47.0,55.0,4.0,28.0,43.0,10.0,0.0,728.0,1742.0,0.0,0.00,29.79,30.48,8.0,35.0,8.2
4,6,2012-01-01,63.0,34.0,49.0,0.0,31.0,43.0,16.0,0.0,727.0,1742.0,0.0,0.00,29.95,30.47,14.0,36.0,13.8
5,7,2012-01-01,50.0,33.0,42.0,0.0,26.0,35.0,23.0,0.0,727.0,1742.0,0.0,0.00,29.15,30.54,10.3,32.0,10.2
6,8,2012-01-01,66.0,45.0,42.0,0.0,34.0,46.0,23.0,0.0,727.0,1742.0,0.0,0.00,30.05,30.54,11.0,36.0,10.9
7,9,2012-01-01,34.0,19.0,27.0,0.0,17.0,23.0,38.0,0.0,727.0,1742.0,0.0,0.00,29.34,30.09,22.8,30.0,22.5
8,10,2012-01-01,73.0,53.0,63.0,0.0,55.0,58.0,2.0,0.0,723.0,1738.0,0.0,0.00,30.16,30.19,5.1,24.0,5.5
9,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,724.0,1737.0,0.0,0.00,30.15,30.18,4.6,23.0,4.8


In [17]:
holidays = get_holidays("holidays.txt")
holiday_names = get_holiday_names("holiday_names.txt")

In [18]:
holidays

,date2
0,2012-01-01
1,2012-01-02
2,2012-01-16
3,2012-02-14
4,2012-02-20
5,2012-04-08
6,2012-05-13
7,2012-05-28
8,2012-06-17
9,2012-07-04


In [19]:
holiday_names

,date2,holiday_name
0,2012-01-01,NewYearsDay
1,2012-01-16,MartinLutherKingDay
2,2012-02-14,ValentinesDay
3,2012-02-20,PresidentsDay
4,2012-04-08,EasterSunday
5,2012-05-13,MothersDay
6,2012-05-28,MemorialDay
7,2012-06-17,FathersDay
8,2012-07-04,IndependenceDay
9,2012-09-03,LaborDay


In [20]:
result = pd.merge(train_file, key_file, how='outer')
result

,date,store_nbr,item_nbr,units,station_nbr
0,2012-01-01,1,1,0,1
1,2012-01-01,1,2,0,1
2,2012-01-01,1,3,0,1
3,2012-01-01,1,4,0,1
4,2012-01-01,1,5,0,1
5,2012-01-01,1,6,0,1
6,2012-01-01,1,7,0,1
7,2012-01-01,1,8,0,1
8,2012-01-01,1,9,29,1
9,2012-01-01,1,10,0,1


In [21]:
holidays = pd.read_csv('data/holidays.csv', sep=',')
holidays

,date,day,holiday_name
0,2012-01-01,Sunday,NewYearsDay
1,2012-01-16,Monday,MartinLutherKingDay
2,2012-02-14,Tuesday,ValentinesDay
3,2012-02-20,Monday,PresidentsDay
4,2012-04-08,Sunday,EasterSunday
5,2012-05-13,Sunday,MothersDay
6,2012-05-28,Monday,MemorialDay
7,2012-06-17,Sunday,FathersDay
8,2012-07-04,Wednesday,IndependenceDay
9,2012-09-03,Monday,LaborDay


In [22]:
set(holidays['holiday_name'])

{'ChristmasDay',
 'ChristmasEve',
 'ColumbusDay',
 'EasterSunday',
 'FathersDay',
 'Halloween',
 'IndependenceDay',
 'LaborDay',
 'MartinLutherKingDay',
 'MemorialDay',
 'MothersDay',
 'NewYearsDay',
 'NewYearsEve',
 'PresidentsDay',
 'ThanksgivingDay',
 'ValentinesDay',
 'VeteransDay'}

In [23]:
t_result = pd.merge(result, holidays, how='left')
t_result

,date,store_nbr,item_nbr,units,station_nbr,day,holiday_name
0,2012-01-01,1,1,0,1,Sunday,NewYearsDay
1,2012-01-01,1,2,0,1,Sunday,NewYearsDay
2,2012-01-01,1,3,0,1,Sunday,NewYearsDay
3,2012-01-01,1,4,0,1,Sunday,NewYearsDay
4,2012-01-01,1,5,0,1,Sunday,NewYearsDay
5,2012-01-01,1,6,0,1,Sunday,NewYearsDay
6,2012-01-01,1,7,0,1,Sunday,NewYearsDay
7,2012-01-01,1,8,0,1,Sunday,NewYearsDay
8,2012-01-01,1,9,29,1,Sunday,NewYearsDay
9,2012-01-01,1,10,0,1,Sunday,NewYearsDay


In [24]:
t_result['date'] = pd.to_datetime(t_result['date'], errors='coerce')
t_result['year'] = t_result['date'].dt.year
t_result['month'] = t_result['date'].dt.month
t_result['day'] = t_result['date'].dt.day
t_result['weekday'] = t_result[['date']].apply(lambda x: dt.datetime.strftime(x['date'], '%a'), axis=1)
t_result

,date,store_nbr,item_nbr,units,station_nbr,day,holiday_name,year,month,weekday
0,2012-01-01,1,1,0,1,1,NewYearsDay,2012,1,Sun
1,2012-01-01,1,2,0,1,1,NewYearsDay,2012,1,Sun
2,2012-01-01,1,3,0,1,1,NewYearsDay,2012,1,Sun
3,2012-01-01,1,4,0,1,1,NewYearsDay,2012,1,Sun
4,2012-01-01,1,5,0,1,1,NewYearsDay,2012,1,Sun
5,2012-01-01,1,6,0,1,1,NewYearsDay,2012,1,Sun
6,2012-01-01,1,7,0,1,1,NewYearsDay,2012,1,Sun
7,2012-01-01,1,8,0,1,1,NewYearsDay,2012,1,Sun
8,2012-01-01,1,9,29,1,1,NewYearsDay,2012,1,Sun
9,2012-01-01,1,10,0,1,1,NewYearsDay,2012,1,Sun


In [25]:
t_result.isnull().sum()

date                  0
store_nbr             0
item_nbr              0
units                 0
station_nbr           0
day                   0
holiday_name    4415025
year                  0
month                 0
weekday               0
dtype: int64

In [26]:
all_data_result = pd.merge(t_result, weather, how='left')
all_data_result

,date,store_nbr,item_nbr,units,station_nbr,day,holiday_name,year,month,weekday,...,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,2012-01-01,1,1,0,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2012-01-01,1,2,0,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
2,2012-01-01,1,3,0,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
3,2012-01-01,1,4,0,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
4,2012-01-01,1,5,0,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
5,2012-01-01,1,6,0,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
6,2012-01-01,1,7,0,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
7,2012-01-01,1,8,0,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
8,2012-01-01,1,9,29,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
9,2012-01-01,1,10,0,1,1,NewYearsDay,2012,1,Sun,...,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6


In [27]:
all_data_result.isnull().sum()

date                  0
store_nbr             0
item_nbr              0
units                 0
station_nbr           0
day                   0
holiday_name    4415025
year                  0
month                 0
weekday               0
tmax                  0
tmin                  0
tavg                  0
depart                0
dewpoint              0
wetbulb               0
heat                  0
cool                  0
sunrise               0
sunset                0
codesum               0
snowfall              0
preciptotal           0
stnpressure           0
sealevel              0
resultspeed           0
resultdir             0
avgspeed              0
dtype: int64

In [28]:
all_data_result.columns

Index(['date', 'store_nbr', 'item_nbr', 'units', 'station_nbr', 'day',
       'holiday_name', 'year', 'month', 'weekday', 'tmax', 'tmin', 'tavg',
       'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset',
       'codesum', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed'],
      dtype='object')

In [29]:
all_data_result.isnull().sum()

date                  0
store_nbr             0
item_nbr              0
units                 0
station_nbr           0
day                   0
holiday_name    4415025
year                  0
month                 0
weekday               0
tmax                  0
tmin                  0
tavg                  0
depart                0
dewpoint              0
wetbulb               0
heat                  0
cool                  0
sunrise               0
sunset                0
codesum               0
snowfall              0
preciptotal           0
stnpressure           0
sealevel              0
resultspeed           0
resultdir             0
avgspeed              0
dtype: int64

In [30]:
all_data_result2 = all_data_result.drop('codesum', axis=1)
all_data_result2

,date,store_nbr,item_nbr,units,station_nbr,day,holiday_name,year,month,weekday,...,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,2012-01-01,1,1,0,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6
1,2012-01-01,1,2,0,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6
2,2012-01-01,1,3,0,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6
3,2012-01-01,1,4,0,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6
4,2012-01-01,1,5,0,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6
5,2012-01-01,1,6,0,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6
6,2012-01-01,1,7,0,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6
7,2012-01-01,1,8,0,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6
8,2012-01-01,1,9,29,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6
9,2012-01-01,1,10,0,1,1,NewYearsDay,2012,1,Sun,...,0,-,-,M,0.05,29.78,29.92,3.6,20,4.6


In [31]:
all_data_result2.columns

Index(['date', 'store_nbr', 'item_nbr', 'units', 'station_nbr', 'day',
       'holiday_name', 'year', 'month', 'weekday', 'tmax', 'tmin', 'tavg',
       'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset',
       'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed',
       'resultdir', 'avgspeed'],
      dtype='object')

In [ ]:
units ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed

In [ ]:
model_1 = sm.OLS.from_formula("units ~ sunrise + sunset + snowfall + preciptotal", data=all_data_result2)
result_model_1 = model_1.fit()
print(result_model_1.summary())